### BERT regressor (norm) 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset, check and remove null values
data_01 = pd.read_csv('Emo_Bank_VAD.csv')
null_rows = data_01[data_01['text'].isnull()]
print(null_rows)
data_01 = data_01.dropna(subset=['text'])
data_01.isnull().sum()

# Normalize columns 'A', 'V', and 'D'
scaler = MinMaxScaler(feature_range=(-1, 1))
data_01[['A', 'V', 'D']] = scaler.fit_transform(data_01[['A', 'V', 'D']])

# Split the data
x_train, x_test, y_train, y_test = train_test_split(data_01["text"], data_01[["V", "A", "D"]], test_size=0.1, shuffle=True, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True, random_state=1)
print("Data shapes:", x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape)


                          id split     V     A     D text
8281  easy_money_13624_13628   dev  2.78  2.89  2.78  NaN
Data shapes: (8022,) (892,) (991,) (8022, 3) (892, 3) (991, 3)


In [8]:
#tokenize and pad
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 170
x_train_pad = tokenizer(x_train.tolist(), padding='max_length', truncation=True, max_length=170, return_tensors="tf")
x_val_pad = tokenizer(x_val.tolist(), padding='max_length', truncation=True, max_length=170, return_tensors="tf")
x_test_pad = tokenizer(x_test.tolist(), padding='max_length', truncation=True, max_length=170, return_tensors="tf")


In [10]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load BERT model 
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the model
def create_model():
    # BERT input/output layers
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    pooled_output = bert_output.pooler_output  

    # Dense layer for regression
    dense = Dense(64, activation='relu')(pooled_output)  
    output = Dense(3, activation='tanh')(dense)  # Output layer for 3 values (V, A, D), using 'tanh' to stay in range [-1, 1]

    # Define the complete model
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    
    return model

# Instantiate the model
model_BERT = create_model()


# Compile the model 
model_BERT.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss='mean_squared_error', 
              metrics=['mae']) 



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
# Train the model 
history = model_BERT.fit(
    x={'input_ids': x_train_pad['input_ids'], 'attention_mask': x_train_pad['attention_mask']},
    y=y_train,
    validation_data=({'input_ids': x_val_pad['input_ids'], 'attention_mask': x_val_pad['attention_mask']}, y_val),
    epochs=6,  
    batch_size=16  
)

# Evaluate on the test set
test_loss, test_mae = model_BERT.evaluate({'input_ids': x_test_pad['input_ids'], 'attention_mask': x_test_pad['attention_mask']}, y_test)
print(f"Test MSE: {test_loss}, Test MAE: {test_mae}")

# Predict on the test set
y_pred = model_BERT.predict({'input_ids': x_test_pad['input_ids'], 'attention_mask': x_test_pad['attention_mask']})

# Compute additional evaluation metrics like MSE and MAE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}, Mean Absolute Error: {mae}")


Epoch 1/6


2024-09-29 13:13:04.836388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


502/502 [==============================] - ETA: 0s - loss: 0.0646 - mae: 0.1869

2024-09-29 13:31:10.426137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


502/502 [==============================] - 1157s 2s/step - loss: 0.0646 - mae: 0.1869 - val_loss: 0.0293 - val_mae: 0.1321
Epoch 2/6
502/502 [==============================] - 1629s 3s/step - loss: 0.0327 - mae: 0.1363 - val_loss: 0.0580 - val_mae: 0.1914
Epoch 3/6
502/502 [==============================] - 1854s 4s/step - loss: 0.0284 - mae: 0.1279 - val_loss: 0.0293 - val_mae: 0.1324
Epoch 4/6
502/502 [==============================] - 2013s 4s/step - loss: 0.0351 - mae: 0.1412 - val_loss: 0.0311 - val_mae: 0.1286
Epoch 5/6
502/502 [==============================] - 2451s 5s/step - loss: 0.0262 - mae: 0.1224 - val_loss: 0.0245 - val_mae: 0.1170
Epoch 6/6
31/31 [==============================] - 136s 4s/step - loss: 0.0292 - mae: 0.1298
Test MSE: 0.029166067019104958, Test MAE: 0.12983328104019165


2024-09-29 16:33:48.712909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 144s 4s/step
Mean Squared Error: 0.02916606507144547, Mean Absolute Error: 0.1298332695289107


In [13]:
# Ensure that y_test is converted to a numpy array
y_test_np = y_test.to_numpy()

# Predict on the tokenized test set using the trained BERT model
y_pred = model_BERT.predict({'input_ids': x_test_pad['input_ids'], 'attention_mask': x_test_pad['attention_mask']})

# Convert predictions to numpy array
y_pred_np = np.array(y_pred)

# Define RMSE function
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Evaluate performance
def evaluate_performance(y_true, y_pred):
    metrics = {}
    metrics['RMSE_V'] = rmse(y_true[:, 0], y_pred[:, 0])
    metrics['RMSE_A'] = rmse(y_true[:, 1], y_pred[:, 1])
    metrics['RMSE_D'] = rmse(y_true[:, 2], y_pred[:, 2])
    
    metrics['MAE_V'] = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    metrics['MAE_A'] = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    metrics['MAE_D'] = mean_absolute_error(y_true[:, 2], y_pred[:, 2])
    
    metrics['MSE_V'] = mean_squared_error(y_true[:, 0], y_pred[:, 0])
    metrics['MSE_A'] = mean_squared_error(y_true[:, 1], y_pred[:, 1])
    metrics['MSE_D'] = mean_squared_error(y_true[:, 2], y_pred[:, 2])
    
    return metrics

# Calculate performance metrics for the test dataset
test_metrics = evaluate_performance(y_test_np, y_pred_np)

# Print evaluation metrics
print("Test Dataset Evaluation:")
print(f'RMSE for V: {round(test_metrics["RMSE_V"], 2)}')
print(f'RMSE for A: {round(test_metrics["RMSE_A"], 2)}')
print(f'RMSE for D: {round(test_metrics["RMSE_D"], 2)}\n')
print(f'MAE for V: {round(test_metrics["MAE_V"], 2)}')
print(f'MAE for A: {round(test_metrics["MAE_A"], 2)}')
print(f'MAE for D: {round(test_metrics["MAE_D"], 2)}\n')
print(f'MSE for V: {round(test_metrics["MSE_V"], 2)}')
print(f'MSE for A: {round(test_metrics["MSE_A"], 2)}')
print(f'MSE for D: {round(test_metrics["MSE_D"], 2)}\n')

# Convert predictions to DataFrame and round to two decimal places
y_pred_df = pd.DataFrame(y_pred_np, columns=["V_pred", "A_pred", "D_pred"]).round(2)

# Concatenate real and predicted values for comparison
test_comparison = pd.concat([y_test.reset_index(drop=True), y_pred_df], axis=1)

# Print comparison table
print("Test Data Comparison:\n", test_comparison.head())

# Save the comparison table to a CSV file
test_comparison.to_csv("test_comparison_BERT_regression_03.csv", index=False)


31/31 [==============================] - 79s 3s/step
Test Dataset Evaluation:
RMSE for V: 0.15
RMSE for A: 0.18
RMSE for D: 0.18

MAE for V: 0.12
MAE for A: 0.14
MAE for D: 0.14

MSE for V: 0.02
MSE for A: 0.03
MSE for D: 0.03

Test Data Comparison:
               V         A         D  V_pred  A_pred  D_pred
0  5.882353e-02 -0.076923  0.008264    0.06   -0.24   -0.04
1  2.220446e-16 -0.076923  0.256198   -0.13   -0.14   -0.13
2 -2.705882e-01  0.092308  0.008264   -0.07    0.03   -0.11
3  2.941176e-01 -0.076923  0.090909   -0.04   -0.02   -0.03
4 -1.176471e-01  0.000000 -0.074380    0.19   -0.07    0.06
